<a href="https://colab.research.google.com/github/ubermenchh/arxiv_chat/blob/main/ArxivChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q llama-index langchain pypdf sentence-transformers sentencepiece arxiv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.3/914.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.5/177.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 7.5 MB/s 

In [2]:
import arxiv

paper = next(arxiv.Client().results(arxiv.Search(id_list=["1706.03762"])))
paper.download_pdf()

'./1706.03762v7.Attention_Is_All_You_Need.pdf'

In [3]:
from llama_index import SimpleDirectoryReader

docs = SimpleDirectoryReader(input_files=[paper.download_pdf()]).load_data()

In [4]:
from llama_index import Document

doc = Document(text='\n\n'.join([doc.text for doc in docs]))

In [5]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from langchain.llms import HuggingFaceHub
from llama_index.llms import LangChainLLM

from google.colab import userdata
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('hf_read_token')

In [6]:
llm = LangChainLLM(llm=HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-beta', model_kwargs={'temperature': 0.3}))
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_documents([doc], service_context=service_context)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
query_engine = index.as_query_engine()

In [12]:
response = query_engine.query('Who are the authors of this paper?')

In [13]:
str(response)

'\n\nThe authors of this paper are Vaswani et al. (2017). Their paper, titled "Attention Is All You Need," presents the Transformer, a sequence transduction model based entirely on attention, replacing recurrent layers commonly used in encoder-decoder architectures with multi-headed self-attention. The paper discusses the theoretical properties of attention mechanisms and provides empirical evidence of their effectiveness. The authors also present experimental results on machine translation tasks, achieving state-of-the-art results on both the WMT 2014 English-to-German and English-to-French translation tasks. Additionally, the paper explores the use of the Transformer for English constituency parsing, demonstrating its ability to generalize to other tasks. The paper is published in the proceedings of the 2017 International Conference on Learning Representations (ICLR).'

In [14]:
from llama_index import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index import load_index_from_storage
import os


def build_sentence_window_index(
    document, llm, embed_model="local:BAAI/bge-small-en-v1.5", save_dir="sentence_index"
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            [document], service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index,
    similarity_top_k=6,
    rerank_top_n=2,
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [20]:
sentence_index = build_sentence_window_index(
    doc,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir='sentence_index'
)

In [21]:
sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [22]:
window_response = sentence_window_engine.query('Who are the authors of this paper?')

In [23]:
str(window_response)

'\n\n[32] Noam Shazeer, Azalia Mirhoseini, Krzysztof Maziarz, Andy Davis, Quoc Le, Geoffrey Hinton,\nand Jeff Dean.\n\n[33] Nitish Srivastava, Geoffrey E Hinton, Alex Krizhevsky, Ilya Sutskever, and Ruslan Salakhutdi-\nnov.\n\nThe authors of paper [32] are Noam Shazeer, Azalia Mirhoseini, Krzysztof Maziarz, Andy Davis, Quoc Le, Geoffrey Hinton, and Jeff Dean. The authors of paper [33] are Nitish Srivastava, Geoffrey E Hinton, Alex Krizhevsky, Ilya Sutskever, and Ruslan Salakhutdi-nov.'

In [24]:
response = query_engine.query('Who are the authors of this paper?')
str(response)

'\n\nThe authors of this paper are Vaswani et al. (2017). Their paper, titled "Attention Is All You Need," presents the Transformer, a sequence transduction model based entirely on attention, replacing recurrent layers commonly used in encoder-decoder architectures with multi-headed self-attention. The paper discusses the theoretical properties of attention mechanisms and provides empirical evidence of their effectiveness. The authors also present experimental results on machine translation tasks, achieving state-of-the-art results on both the WMT 2014 English-to-German and English-to-French translation tasks. Additionally, the paper explores the use of the Transformer for English constituency parsing, demonstrating its ability to generalize to other tasks. The paper is published in the proceedings of the 2017 International Conference on Learning Representations (ICLR).'